In [2]:
from moleculekit.molecule import Molecule
from moleculekit.projections.metricrmsd import MetricRmsd
import numpy as np
import os

## Create levels with multiple starting conformations

In [ ]:
import copy

min_rmsd = 0
max_rmsd = 1
stop = False
level = 0
rmsds_dict = {}

while stop == False:
    stop = True
    for i in range(64):
        mol1 = copy.deepcopy(mol)
        mol1.dropFrames(keep=[i])


        mol1.align('name CA', refmol=ref_mol)
        rmsd_proj = MetricRmsd(ref_mol, 'name CA', centerstr='name CA', pbc=False)
        rmsd = rmsd_proj.project(mol1)

        if min_rmsd <= rmsd <= max_rmsd:
            rmsds_dict['level'+str(level + 1)] = rmsd.item()
            print(rmsd)
            min_rmsd += 1
            max_rmsd += 1
            stop = False       
            path = '/workspace7/torchmd-AD/train_curriculum/cln/levels/level' + str(level)
            #os.mkdir(path)
            mol1.write(os.path.join(path, 'chignolin_cln025.xtc' ))
            level += 1
    
    

0.8739174
1.4634277
2.8587408
3.779713
4.0096207
5.7043376
6.3186293


In [96]:
import pandas as pd
pd.DataFrame.from_dict(rmsds_dict, orient='index', columns = ['starting rmsd'])

,starting rmsd
level1,0.873917
level2,1.463428
level3,2.858741
level4,3.779713
level5,4.009621
level6,5.704338
level7,6.318629


In [120]:
mol_curriculum.write('/workspace7/torchmd-AD/train_curriculum/cln/psf/chignolin_cln025.psf')
mol_curriculum.write('/workspace7/torchmd-AD/train_curriculum/cln/xtc/chignolin_cln025.xtc')

## Create levels with multiple starting conformations

In [38]:
# Reference Molecule
psf = f'../data/bba/bba_ca_top_dih.psf'
ref_pdb = f'../data/bba/bba_ca.pdb'
ref_mol = Molecule(psf)
ref_mol.read(ref_pdb)

# Read the array with trajectory data

mol = Molecule(psf)
arr = np.load('/workspace2/fast_folders_cgnet/multiprotein/data_dih/bba_ca_coords.npy')[:1000000]
mol.coords = np.moveaxis(arr, 0, -1)

In [39]:
# Compute the rmsd of each frame against the reference structure
mol.align('name CA', refmol=ref_mol)
rmsd_proj = MetricRmsd(ref_mol, 'name CA', centerstr='name CA', pbc=False)
rmsd = rmsd_proj.project(mol)

In [41]:
# Sample frames for different levels and save them in levels folders
from statistics import mean
n_levels = int(1 // 1)
min_rmsd = 0
max_rmsd = 2

path = '/workspace7/torchmd-AD/train_curriculum/bba/levels/level'

for n in range(n_levels):
    
    indexes = np.where(np.logical_and(rmsd>=min_rmsd, rmsd<=max_rmsd))[0]
    frames = arr[indexes, :, :][:200]
    
    print(mean(rmsd[indexes]))
    
    directory = path + str(n)
    os.mkdir(directory)
    
    np.save(os.path.join(directory, 'bba_ca.npy'), frames)
    min_rmsd += 1
    max_rmsd += 1
    
    

1.9022334


In [44]:
rmsd[np.where(rmsd < 2.0)]

array([1.9836608, 1.9691807, 1.902709 , 1.879754 , 1.7387888, 1.9393072],
      dtype=float32)